# Dowload APK file from https://apkcombo.com/

In [1]:
import os
import time
import glob
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import requests
import pandas as pd
import os
import urllib.parse
import threading
from time import sleep
from threading import Thread
import glob

In [2]:
def deleteFile(dir):
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))

In [3]:
def waitingDownload():
    path = "C:\\Users\\ASUS\\anaconda3\\phd_implement\\download_apk\\tmp\\*.apk"
    path_xapk = "C:\\Users\\ASUS\\anaconda3\\phd_implement\\download_apk\\tmp\\*.xapk"
    files = glob.glob(path)
    files_xapk = glob.glob(path_xapk)
    #print(files)
    if(len(files)!=0):
        return True
    elif(len(files_xapk)!=0):
        return True
    else:
        return False

In [4]:
def fileAPKPathFunc():
    path = "C:\\Users\\ASUS\\anaconda3\\phd_implement\\download_apk\\tmp\\*.apk"
    path_xapk = "C:\\Users\\ASUS\\anaconda3\\phd_implement\\download_apk\\tmp\\*.xapk"
    files = glob.glob(path)
    files_xapk = glob.glob(path_xapk)
    #print(files)
    if((len(files)!=0)):
        return files[0],"apk"
    elif((len(files_xapk)!=0)):
        return files_xapk[0],"xapk"
    else:
        return "",""

In [5]:
def removeInvalidCharacter(filename):
    invalid = '<>:"/\|?*!+;,='
    for char in invalid:
        filename = filename.replace(char,'_')
    return filename

In [6]:
# 0. Read apk name
df=""
downloaded_index = 0
# 0.1 Read file
#file_exists = os.path.exists("C:\\Users\\ASUS\\anaconda3\\phd_implement\\app_category.csv") // grouping
file_exists = os.path.exists("C:\\Users\\ASUS\\anaconda3\\phd_implement\\grouping_apk.csv")
print(file_exists)
if(file_exists):
    #df = pd.read_csv("C:\\Users\\ASUS\\anaconda3\\phd_implement\\app_category.csv",low_memory=False)
    df = pd.read_csv("C:\\Users\\ASUS\\anaconda3\\phd_implement\\grouping_apk.csv",low_memory=False)
    # 0.2 Get next download index
    length = len(df["download"])
    print(length)
    for i in range(len(df["download"])):
        if((df["download"][i]!="Done") and (df["download"][i]!="Not Found")):
            print(i)
            downloaded_index = i
            break
    print(downloaded_index)
else:
    df = pd.read_csv("C:\\Users\\ASUS\\anaconda3\\phd_implement\\download_apps.csv",low_memory=False)
pkg_name = df["pkg_name"]

True
19591842
5543
5543


In [7]:
# 1. Setup Chrome driver environement
# 1.1 Setup selenium
chromedriver = 'chromedriver.exe'
os.environ["webdriver.chrome.driver"] = chromedriver
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument('--ignore-ssl-errors=yes')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--no-proxy-server')
prefs = {
    'profile.default_content_setting_values.automatic_downloads': 1,
    "download.default_directory" : "C:\\Users\\ASUS\\anaconda3\\phd_implement\\download_apk\\tmp\\"
}
chrome_options.add_experimental_option("prefs", prefs)

In [8]:
step = 20
for i in range(downloaded_index,downloaded_index+step):   
    # 2. Load web page
    # 2.1 Open web page
    web = webdriver.Chrome(options=chrome_options)
    web.maximize_window()
    web.get("https://apkcombo.com")
    time.sleep(7)
    # 2.2 Agree privacy
    try:
        privacy_agree=web.find_element(By.CSS_SELECTOR,"button[class='sc-ifAKCX ljEJIv']")
        privacy_agree.click()
        time.sleep(5)
    except NoSuchElementException:
        print("Element does not exist")
    # 3. Find textbox and input apk name
    # 3.1 Find text box
    apk_name_input=web.find_element(By.NAME, 'q')
    #print(apk_name_input)
    # 3.2 Input apk name
    #search_apk=apk_name_input.send_keys("com.facebook.katana")
    search_apk=apk_name_input.send_keys(pkg_name[i])
    # 4. Search apk
    time.sleep(10)
    search=web.find_element(By.CSS_SELECTOR,"button[class='button button-search is-link']")
    search.click()
    time.sleep(5)
    # 5. Download APK
    # 5.1 Get current url
    current_url = web.current_url
    web.close()
    # 5.2 Load download url
    download_url = current_url+"download/apk"
    #print(download_url)
    # 5.3 Open web page
    web = webdriver.Chrome(options=chrome_options)
    web.maximize_window()
    web.get(download_url)
    time.sleep(10)
    # 5.4 Agree privacy
    try:
        privacy_agree=web.find_element(By.CSS_SELECTOR,"button[class='sc-ifAKCX ljEJIv']")
        privacy_agree.click()
        time.sleep(10)
    except NoSuchElementException:
        print("Element does not exist")
    # 5.5 Find download link 
    try:
        apk_link=web.find_element(By.CLASS_NAME, "variant").get_attribute('href')
        apkfile_name = web.find_element(By.CLASS_NAME, "vername").text
        print("[Link-"+str(i)+"] "+apk_link)
        apkfile_name=removeInvalidCharacter(apkfile_name)
        df.loc[i,"apkfile_name"]=str(apkfile_name+".apk")
    except NoSuchElementException:
        print("[Link-"+str(i)+"] App does not exist")
        df.loc[i,"download"]="Not Found"
        continue
    # 5.6 Download APK files from download_link
    #web.get(apk_link)
    def downloadAPK():
        web.get(apk_link)
    thread= Thread(target=downloadAPK())
    thread.start()
    print('Waiting for the apk downloading...')
    while True:
        if(waitingDownload()):
            break
    thread.join()
    print('Download completed')
    time.sleep(15)
    df.loc[i,"download"]="Done"
    fileAPKPath,fileExtension=fileAPKPathFunc()
    if((fileAPKPath!="") and (fileExtension!="")):
        os.rename(fileAPKPath, "C:\\Users\\ASUS\\anaconda3\\phd_implement\\download_apk\\"+str(apkfile_name)+"."+str(fileExtension))
        time.sleep(15)
        deleteFile("C:\\Users\\ASUS\\anaconda3\\phd_implement\\download_apk\\tmp\\")
        time.sleep(15)
    time.sleep(5)
    # 6. Filter APK to category
    # 6.1 Get APK category
    breadcrumb = web.find_element(By.CSS_SELECTOR,"nav[class='breadcrumb']")
    breadcrumb_item = breadcrumb.find_elements(By.TAG_NAME, "a")
    category_link = []
    app_name = []
    for j in breadcrumb_item:
        category_link.append(j.get_attribute('href'))
    apk_category = str((category_link[2].split("/"))[4])
    app_name = str((category_link[3].split("/"))[3])
    print("APK File Name: "+apkfile_name+".apk")
    print("App Category: " +apk_category)
    print("App Name: " +app_name)
    # 6.2 Update Category
    df.loc[i,"apk_category"]=apk_category
    df.loc[i,"app_name"]=app_name
    time.sleep(5)
    web.close()

[Link-5543] App does not exist
[Link-5544] https://download.apkcombo.com/com.grinch.videocall.livechat/The%20Grinch%20call%20%E2%98%8E%EF%B8%8F%20Grinch%20Video%20Call%20and%20Live%20Chat_1.0.1_apkcombo.com.apk?ecp=Y29tLmdyaW5jaC52aWRlb2NhbGwubGl2ZWNoYXQvMS4wLjEvMy41ZTkwZjUzZjE5NDExZTYxZGZjZjA0NTM1MGNmMzQxMTIzMGYyZTc3LmFwaw==&iat=1683731391&sig=ad293a9cb104ce38103ab147e4c6e91c&size=80576683&from=cf&version=latest&lang=en&fp=de1d934c1351016dab2b207ce49f7ab5&ip=188.152.79.104
Waiting for the apk downloading...
Download completed
APK File Name: The Grinch call ☎️ Grinch Video Call and Live Chat 1.0.1.apk
App Category: entertainment
App Name: the-grinch-call-%E2%98%8E%EF%B8%8F-grinch-video-call-and-live-chat
[Link-5545] https://download.apkcombo.com/com.vertaler.deit/German%20Italian%20Translator_3.3.5_apkcombo.com.apk?ecp=Y29tLnZlcnRhbGVyLmRlaXQvMy4zLjUvNzIuMzJjMDFhZjk0YTVlZDAyZDMzNjc5YWQ2MjQwODkzMWIxMTlmMGQ4YS5hcGs=&iat=1683731521&sig=3f8b076e859547dace1214cc598b284d&size=4646868&from=cf

Waiting for the apk downloading...
Download completed
APK File Name: Amino para Resident Evil 3.4.33514.apk
App Category: social
App Name: amino-para-resident-evil
[Link-5560] https://apkcombo.com/d?u=aHR0cHM6Ly9kLTA5LndpbnVkZi5jb20vYi9BUEsvWTI5dExtRm1abWw0WTJoaGRDNWphR0YwWVhCd2RqQmZNMTgxT0RBeU1XRm1Zdz9rPWNmOTNhYmQ0ODY2ZDA5ODQ0ZTVjYTcyZDE0NmMzNTU0NjQ1ZTVkOGQmX2ZuPVFXWm1hWGdyWTJoaGRGOHhMakV1TVY5aGNHdGpiMjFpYnk1amIyMHVZWEJy&fp=de1d934c1351016dab2b207ce49f7ab5&ip=188.152.79.104
Waiting for the apk downloading...
Download completed
APK File Name: Affix chat 1.1.1.apk
App Category: communication
App Name: affix-chat
[Link-5561] https://download.apkcombo.com/com.spark.mahalaxmidesign/Embroidery%20Design%20-%20Mahalaxmi%20Design_7.0.0.31_apkcombo.com.apk?ecp=Y29tLnNwYXJrLm1haGFsYXhtaWRlc2lnbi83LjAuMC4zMS8zNS4xOTIxNjBiZDZjYjJjMzY1MDQ3ZWIwOTYxZWU2ODc1MzY5MjYyNzBkLmFwaw==&iat=1683733306&sig=e689ba5601e0b17eda3f6a3967db8bd5&size=13123360&from=cf&version=latest&lang=en&fp=de1d934c1351016dab2b207ce

In [9]:
# 7. Export CSV with category
df.to_csv("grouping_apk.csv",index=False)
export = pd.read_csv("grouping_apk.csv",low_memory=False)
export[downloaded_index:downloaded_index+step]

,pkg_name,download,apkfile_name,apk_category,app_name,classify
5543,com.Earn.Tether.Faucet,Not Found,NaN,NaN,NaN,NaN
5544,com.grinch.videocall.livechat,Done,The Grinch call ☎️ Grinch Video Call and Live ...,entertainment,the-grinch-call-%E2%98%8E%EF%B8%8F-grinch-vide...,NaN
5545,com.vertaler.deit,Done,German Italian Translator 3.3.5.apk,education,german-italian-translator,NaN
5546,com.gau.go.launcherex.theme.pinkhazelaunchertheme,Done,Pink Haze Launcher Theme 1.264.1.40.apk,personalization,pink-haze-launcher-theme,NaN
5547,com.mobincube.app_1589851444.sc_5LLYN3,Not Found,NaN,NaN,NaN,NaN
5548,com.iutilities.HSPAP.Optimizer,Done,HSPA_ _ H_ Signal Optimizer 6.5.apk,tools,hspa-h-signal-optimizer,NaN
5549,com.larwoodvoce.app,Done,Larwood & Voce 4.09.058.apk,food-and-drink,larwood-voce,NaN
5550,com.asw.www,Done,FigitalBox 2.2.apk,productivity,figitalbox,NaN
5551,com.ordersnapp.taquerialahacienda,Not Found,NaN,NaN,NaN,NaN
5552,de.consoft.oventrop.ovapp,Done,Oventrop 4.42.9.apk,tools,oventrop,NaN
